In [1]:
import sys
sys.path.append('../src/')

from data_loader import DataLoader
from mnist import load_mnist
from model import Model
from multilayer_perceptron import MultilayerPerceptron as MLP, SGDTrainer
from random_fns import Normal, RandomFunction
from supervisor import Supervisor

In [2]:
train_data, val_data, _ = load_mnist('../data/mnist.pkl.gz')

train_dataloader = DataLoader(train_data, batch_size=10)
val_dataloader = DataLoader(val_data)

In [3]:
nn = MLP(
    layer_sizes=[784, 15, 10],
    random_fn=Normal(std=1)
)

trainer = SGDTrainer(
    learning_rate=0.01,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader
)
supervisor = Supervisor(
    model=nn,
    trainer=trainer,
    epsilon=0.01,
    early_stopping_threshold=0.0002
)

supervisor()

loss = 0.29, acc = 91.71%
loss = 0.24, acc = 93.02%
loss = 0.22, acc = 93.91%
loss = 0.21, acc = 94.24%
loss = 0.20, acc = 94.19%
Reached the early stopping threshold (0.05%)


4